## Imports

In [9]:
import pandas as pd
import numpy as np
import wfdb
import ast

from helpers import load_raw_data

## Load data

In [10]:
#path = '../input/ptb-xl-dataset/ptb-xl-a-large-publicly-available-electrocardiography-dataset-1.0.1/'
path = 'ptb-xl_ecg/'

Y = pd.read_csv(path + 'ptbxl_database.csv', index_col='ecg_id')
Y.scp_codes = Y.scp_codes.apply(lambda x: ast.literal_eval(x))

X = load_raw_data(df = Y, path=path)

100%|██████████| 21837/21837 [09:21<00:00, 38.87it/s]
